In [1]:
# Import modules
import ee
import json
import requests
import matplotlib.pyplot as plt
from matplotlib import colors
import xarray as xr
import pandas as pd
import geopandas as gpd

In [2]:
# Authenticate
#ee.Authenticate()

# Initialize API
ee.Initialize()

In [ ]:
# Import boundary for area of interest (aoi)
with open('/home/alejandro/Documentos/Lucila-GIS/28.geojson') as file:
    aoi_json = json.load(file)

# Define the ee.Geometry
aoi = ee.Geometry(aoi_json['features'][0]['geometry'])

# Create mask for field
mask = ee.Image.constant(1).clip(aoi).mask()

# Define start and end dates
# Summer for southern hemisphere, matching the soybean growing season
start_date = '2024-05-14'
end_date = '2024-05-16'

In [3]:
# Import boundary for area of interest (aoi)
with open('/home/alejandro/Documentos/Lucila-GIS/20.geojson') as file:
    aoi_json = json.load(file)

# Define the ee.Geometry
aoi = ee.Geometry(aoi_json['features'][0]['geometry'])

# Create mask for field
mask = ee.Image.constant(1).clip(aoi).mask()

# Define start and end dates
# Summer for southern hemisphere, matching the soybean growing season
start_date = '2024-05-14'
end_date = '2024-05-16'

In [4]:
# Load Sentinel-2 image collection
S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterDate(start_date, end_date) \
                .filterBounds(aoi) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .select(['B8', 'B4'])  # B8 is NIR, B4 is Red

# Print the following line to explore the selected images
S2.getInfo()['features']

[{'type': 'Image',
  'bands': [{'id': 'B8',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [10980, 10980],
    'crs': 'EPSG:32720',
    'crs_transform': [10, 0, 600000, 0, -10, 6700000]},
   {'id': 'B4',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [10980, 10980],
    'crs': 'EPSG:32720',
    'crs_transform': [10, 0, 600000, 0, -10, 6700000]}],
  'version': 1718803684516416,
  'id': 'COPERNICUS/S2_HARMONIZED/20240515T135659_20240515T140402_T20JPM',
  'properties': {'DATATAKE_IDENTIFIER': 'GS2B_20240515T135659_037559_N05.10',
   'SPACECRAFT_NAME': 'Sentinel-2B',
   'RADIO_ADD_OFFSET_B8A': -1000,
   'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 101.986891400441,
   'RADIO_ADD_OFFSET_B10': -1000,
   'MEAN_SOLAR_AZIMUTH_ANGLE': 31.9549576967618,
   'system:footprint': {'type': 'LinearRing',
    'coordinates': [[-61.70431504106892, -29.824628795866147],
     [-

In [5]:
def calculate_ndvi(image):
    """
    Function to calculate NDVI for a single image.
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Apply the NDVI function to each image in the collection
ndvi_collection = S2.map(calculate_ndvi)

In [6]:
# Apply the function to add mean NDVI as a property
ndvi_mean = ndvi_collection.reduce(ee.Reducer.mean())

# Mask ndvi image
ndvi_mean = ndvi_mean.mask(mask)

In [7]:
# Get url link for image
image_url = ndvi_mean.getDownloadUrl({'region': aoi,'scale':10,
                                      'bands':['NDVI_mean'],
                                     'crs': 'EPSG:4326', 
                                     'format': 'GEO_TIFF'})
    
# Request data using URL and save data as a new GeoTiff file
response = requests.get(image_url)

# Save geotiff image to local drive
filename = '/home/alejandro/Documentos/Lucila-GIS/20.tif'
with open(filename, 'wb') as f:
    f.write(response.content)